# Environment

In [1]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import random
import threading
import multiprocessing as mp

In [2]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [3]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1614565773.0
Print triggered 1614565773.0
Main triggered 1614565773.5
Main triggered 1614565774.0
Print triggered 1614565774.0


# Controller Base Class (eFMU)

In [4]:
# Test controller
testcontroller = testcontroller1() #this controller multiplies the inputs
# Get all variables
variables = testcontroller.get_model_variables()
# Makeup some inputs
inputs = {}
for var in variables:
    inputs[var] = random.randint(1,50)
# Query controller
print ('Log-message', testcontroller.do_step(inputs=inputs))
print ('Input', testcontroller.input)
print ('Output', testcontroller.output)
print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 7, 'b': 4}
Output {'c': 28}
Output {'c': 28}


# Controller Stack Class (single-thread/multi-thread)

In [5]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [6]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
for i in range(6):
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

Exception ignored in: <function _ConnectionBase.__del__ at 0x7fe6d3d9eaf0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/connection.py", line 137, in __del__
    self._close()
KeyboardInterrupt: 


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:41.980774912,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:44.261884928,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:46.543813120,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:48.821138944,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:51.100631040,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:41.980774912,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:44.261884928,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:46.543813120,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:48.821138944,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:51.100631040,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:41.980774912,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:44.261884928,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:46.543813120,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:48.821138944,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:51.100631040,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:41.980774912,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:44.261884928,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:46.543813120,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:48.821138944,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:51.100631040,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:41.980774912,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:44.261884928,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:46.543813120,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:48.821138944,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:51.100631040,30.0,4.0,120.0,testcontroller1 did a computation!


13.827431201934814


## Multi Thread

In [7]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)


# Call query_control 6 times. We should expect there are 6 records(excluding NaN) for each task.
# In multi thread mod, each call of query_control will trigger a computation for one controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
t = time.time()
while time.time() - t < 100:
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

Controller timeout forecast2


/Users/deoxi/FMLC/FMLC/stackedclasses.py:300: Warning: Controller forecast2 timeout
  print('Controller timeout', name)


Controller timeout forecast2
Controller timeout forecast2
Controller timeout control1


/Users/deoxi/FMLC/FMLC/stackedclasses.py:300: Warning: Controller control1 timeout
  print('Controller timeout', name)


Controller timeout forecast2


...

,a,b,c,Logging
2021-02-28 19:46:01.996430080,NaN,NaN,NaN,Initialize
2021-02-28 19:52:45.658352128,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 19:52:46.930033920,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 19:52:48.180577024,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 19:52:49.421455872,40.0,10.0,400.0,testcontroller2 did a computation!
...,...,...,...,...
2021-02-28 19:54:16.037940992,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 19:54:19.721678080,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 19:54:22.002036992,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 19:54:23.309943040,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-28 19:46:01.996430080,NaN,NaN,NaN,Initialize
2021-02-28 19:52:45.658352128,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 19:52:46.930033920,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 19:52:48.180577024,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 19:52:49.421455872,400.0,40.0,16000.0,testcontroller3 did a computation!
...,...,...,...,...
2021-02-28 19:54:16.037940992,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 19:54:19.721678080,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 19:54:22.002036992,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 19:54:23.309943040,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-28 19:46:01.996430080,NaN,NaN,NaN,Initialize
2021-02-28 19:52:45.658352128,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 19:52:46.685186048,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 19:52:47.715365120,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 19:52:48.747813888,20.0,4.0,80.0,testcontroller3 did a computation!
...,...,...,...,...
2021-02-28 19:54:16.249362944,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 19:54:19.721678080,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 19:54:21.972133888,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 19:54:23.309943040,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-28 19:46:01.996430080,NaN,NaN,NaN,Initialize
2021-02-28 19:52:45.658352128,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 19:52:46.662597888,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 19:52:47.666170112,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 19:52:48.667670016,30.0,4.0,120.0,testcontroller1 did a computation!
...,...,...,...,...
2021-02-28 19:54:16.074288128,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 19:54:19.721678080,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 19:54:21.972133888,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 19:54:23.309943040,30.0,4.0,120.0,testcontroller1 did a computation!


100.26732110977173


In [5]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)
print("done initializing")

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
while time.time() - t <= 100:
    threading.Thread(target=controller_stack.query_control, args=(ctrl_stack, time.time()), daemon=True).start()
print(time.time() - t)

done initializing
Controller timeout forecast2


/Users/deoxi/FMLC/FMLC/stackedclasses.py:301: Warning: Controller forecast2 timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


Controller timeout forecast1


/Users/deoxi/FMLC/FMLC/stackedclasses.py:301: Warning: Controller forecast1 timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


Controller timeout forecast3


...

Controller timeout mpc1


/Users/deoxi/FMLC/FMLC/stackedclasses.py:301: Warning: Controller mpc1 timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


Controller timeout control1


/Users/deoxi/FMLC/FMLC/stackedclasses.py:301: Warning: Controller control1 timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


RuntimeError: can't start new thread

In [6]:
for df in ctrl_stack.log_to_df().values():
    display(df)

,a,b,c,Logging
2021-02-28 20:22:55.842193920,NaN,NaN,NaN,Initialize
2021-02-28 20:24:07.974312960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 20:24:09.406247936,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 20:24:10.776039936,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 20:24:12.197696000,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 20:24:13.587581952,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 20:24:14.970813952,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 20:24:16.391065088,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 20:24:17.733344000,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 20:24:19.124190976,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-28 20:22:55.842193920,NaN,NaN,NaN,Initialize
2021-02-28 20:24:07.974312960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 20:24:09.406247936,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 20:24:10.776039936,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 20:24:12.197696000,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 20:24:13.587581952,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 20:24:14.970813952,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 20:24:16.391065088,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 20:24:17.733344000,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 20:24:19.124190976,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-28 20:22:55.842193920,NaN,NaN,NaN,Initialize
2021-02-28 20:24:07.974312960,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 20:24:09.406247936,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 20:24:10.776039936,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 20:24:12.197696000,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 20:24:13.587581952,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 20:24:14.970813952,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 20:24:16.391065088,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 20:24:17.733344000,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 20:24:19.124190976,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-28 20:22:55.842193920,NaN,NaN,NaN,Initialize
2021-02-28 20:24:07.974312960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 20:24:09.085072896,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 20:24:10.191282944,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 20:24:11.265850112,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 20:24:12.315554048,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 20:24:13.367951872,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 20:24:14.426187008,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 20:24:15.484741120,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 20:24:16.556934912,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-28 20:22:55.842193920,NaN,NaN,NaN,Initialize
2021-02-28 20:24:07.974312960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 20:24:08.974902016,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 20:24:09.976568064,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 20:24:10.981687040,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 20:24:11.991877120,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 20:24:12.994907904,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 20:24:13.995321088,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 20:24:14.995823104,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 20:24:15.997347072,30.0,4.0,120.0,testcontroller1 did a computation!


In [9]:
import time
import concurrent.futures
def f(s):
    time.sleep(s)
    return s
e = concurrent.futures.ThreadPoolExecutor(4)
s = range(10)
for i in e.map(f, s):
    print(i)

0
1
2
3
4
5
6
7
8
9
